#### Imports

In [7]:
import pandas as pd
import numpy as np  
import os
import matplotlib.pyplot as plt
import math
from PIL import Image, ImageOps

from skimage import morphology
from skimage import io
from skimage.segmentation import slic, mark_boundaries

#from os.path import exists

#from pathlib import Path




# Default packages for the minimum example
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GroupKFold
from sklearn.metrics import accuracy_score #example for measuring performance


import pickle #for saving/loading trained classifiers

#### Data

In [8]:
file_data = 'data/metadata.csv'
path_image =  'data/images'
path_mask = 'data/masks'
path_masked_image = 'data/masked_images'
file_features = 'features/features.csv'

images_id = os.listdir(path_image)
images_id.remove('.gitkeep')

#Read meta-data into a Pandas dataframe
metadata = pd.read_csv(file_data)

# Extract image IDs and labels from the data. 
#image_id = list(metadata['img_id'])
#label = np.array(metadata['diagnostic'])

images_id


['PAT_106_159_325.png',
 'PAT_107_160_609.png',
 'PAT_108_162_660.png',
 'PAT_115_1138_870.png',
 'PAT_117_179_983.png',
 'PAT_118_180_386.png',
 'PAT_134_201_587.png',
 'PAT_1420_1460_733.png',
 'PAT_166_3672_759.png',
 'PAT_168_1486_97.png',
 'PAT_183_283_495.png',
 'PAT_194_298_945.png',
 'PAT_1971_3984_534.png',
 'PAT_207_3951_140.png',
 'PAT_216_331_678.png',
 'PAT_224_1340_551.png',
 'PAT_236_361_180.png',
 'PAT_241_367_374.png',
 'PAT_241_367_89.png',
 'PAT_249_380_366.png',
 'PAT_256_295_583.png',
 'PAT_263_404_410.png',
 'PAT_265_406_276.png',
 'PAT_26_37_865.png',
 'PAT_270_1382_561.png',
 'PAT_270_1386_755.png',
 'PAT_276_425_539.png',
 'PAT_281_1516_753.png',
 'PAT_286_1458_381.png',
 'PAT_286_1459_926.png',
 'PAT_288_441_312.png',
 'PAT_289_1399_481.png',
 'PAT_300_648_84.png',
 'PAT_302_1633_845.png',
 'PAT_302_651_529.png',
 'PAT_304_4186_458.png',
 'PAT_306_1646_903.png',
 'PAT_319_680_832.png',
 'PAT_333_1468_620.png',
 'PAT_333_1469_499.png',
 'PAT_340_714_68.png',
 '

### Feature measurements functions

In [5]:
def create_masked_image(path_image, path_mask, path_masked_image, img_filename):
    """
    This function takes in the paths to the images folder, masks folder and 
    masked images folder, and the filename of the image to be masked, and creates 
    a masked image for the specified image using the corresponding mask in the masks folder.
    """
    # Create the masked images folder if it does not exist
    if not os.path.exists(path_masked_image):
        os.makedirs(path_masked_image)

    # Loop through the images folder
    for filename in os.listdir(path_image):
        if filename == img_filename and filename.endswith('.png'): # adjust file extension as needed
            img_path = os.path.join(path_image, filename)
            mask_filename = f"mask_{filename}"
            mask_path = os.path.join(path_mask, mask_filename)
            # Load the image and mask
            img = Image.open(img_path)
            mask = Image.open(mask_path).convert('L')
            # Apply the mask on the image
            masked_img = Image.composite(img, Image.new('RGB', img.size, (0, 0, 0)), mask)
            # Save the masked image
            masked_filename = f"masked_{filename}"
            masked_img.save(os.path.join(path_masked_image, masked_filename))
            return # exit the function after creating the masked image

    # If the specified image filename was not found
    print(f"Error: image {img_filename} not found in {path_image}")

# Loop through the images folder and create a masked image for each image
for filename in os.listdir(path_image):
    create_masked_image(path_image, path_mask, path_masked_image, filename)

# PAT_672_1272_705 <> faulty image, because of extra space in the name

Error: image .gitkeep not found in data/images


In [9]:
''' ASYMMETRY '''  #Okkkk

def fix_mask(mask):
    '''Turns a mask into only black(0) or white(255) pixel values. Put the leison in the
     center of the image '''

    #Make it purely black and white
    bw = np.asarray(mask).copy()
    bw[bw <255] = 0
    mask = Image.fromarray(bw)
    
    #Crop the leison 
    mask_bw = np.where(np.array(mask)==255, 1, 0) #2d array with 0 on black and 1 on white

    row_index =  np.where(np.sum(mask_bw, axis=1)>0)[0] #all the rows with at least one white element 
    first_row , last_row = row_index[0] , row_index[-1]  #first and last row 
    if (last_row - first_row) %2 != 0:
        last_row += 1 #one extra row to make it even and able to halve it

    col_index =  np.where(np.sum(mask_bw, axis=0)>0)[0] #all the col with at least one white element 
    first_col , last_col = col_index[0] , col_index[-1]  #first and last col 
    if (last_col - first_col) %2 != 0:
        last_col += 1

    cropped_mask = mask.crop((first_col,first_row,last_col,last_row))

    #Add borders
    old_width , old_height = cropped_mask.size 
    fixed_mask = ImageOps.expand(cropped_mask, border = int(old_width/2))

    return fixed_mask


def test_asymmetry(mask):
    '''Takes a mask image, halves it vertically, compares both sides. Returns an index of asymmetry 
    as the proportion of leisure that differs on both sides over the total leison'''

    width,height = mask.size

    #Cut in half
    left = mask.crop((0, 0, int(width/2), height)) #left part of picture (left, top, right, bottom)
    right = mask.crop((int(width/2), 0, width, height)) #right part of picture
    right = right.transpose(Image.FLIP_LEFT_RIGHT) #flip right part to compare

    #Compairing both sides
    asym = np.sum(np.where(np.array(left) != np.array(right), 1, 0))
    total_white = np.sum(np.where(np.array(mask)==255, 1, 0))
   
    return round((asym/total_white), 3) #Maybe *10 ??? Very small number 


def get_asymmetry(mask):
    '''Returns the asymmetry for a given leison by rotating the mask image by several angles, measuring 
     the proportion of asymmetry on each, and returning the minimum index. '''
    
    #Leison in the center of the image. Expand black borders to give freedom when rotating image. 
    mask = fix_mask(mask)

    asym = [test_asymmetry(mask.rotate(angle)) for angle in [0,15,30,45,60,75,90]]

    return round(np.min(asym), 3)




In [10]:
def get_color_variability(masked_image, measure='variance'):
    # Find the non-black pixels (i.e., the lesion pixels)
    non_black_pixels = np.where(np.any(masked_image > 0, axis=-1))

    # Extract the color values of the non-black pixels
    r = masked_image[non_black_pixels][:, 0]
    g = masked_image[non_black_pixels][:, 1]
    b = masked_image[non_black_pixels][:, 2]

    # Divide the lesion pixels into segments of similar color
    segments_slic = slic(masked_image[non_black_pixels], n_segments=50, compactness=10, sigma=3, start_label=1)

    # Compute the color variability for each segment that is within the lesion
    segment_color_variability = []
    for i in range(1, np.max(segments_slic) + 1):
        segment_pixels = np.where(segments_slic == i)[0]
        segment_r = r[segment_pixels]
        segment_g = g[segment_pixels]
        segment_b = b[segment_pixels]

        if measure == 'variance':
            rgb_variability = (np.var(segment_r), np.var(segment_g), np.var(segment_b))
        elif measure == 'standard_deviation':
            rgb_variability = (np.std(segment_r), np.std(segment_g), np.std(segment_b))
        else:
            return None

        segment_color_variability.append(np.mean(rgb_variability))

    return np.mean(segment_color_variability)


In [11]:
####### BORDER #########

def area_perimeter(mask): 
    '''Measures the area and perimeter of a mask image'''

    mask = np.array(mask)
    mask = np.where(mask==255, 1, 0)

    #area: the sum of all white pixels in the mask image
    area = np.sum(mask)

    #perimeter: first find which pixels belong to the perimeter.
    struct_el = morphology.disk(1)
    mask_eroded = morphology.binary_erosion(mask, struct_el)
    image_perimeter = mask - mask_eroded
    perimeter = np.sum(image_perimeter)

    return area, perimeter

 
def get_compactness(mask):
    '''Computes and returns the compactness of a figure'''

    area, perimeter = area_perimeter(mask)

    return round( (perimeter ** 2) /(4* np.pi *area), 4)


### Measuring features ---> csv

In [130]:
''''For quick testing of functions. Delete after '''

data = {'images_id': images_id, 'asym': [],'area/perimeter': [] ,'compactness' : []}

for i, image in enumerate(images_id):

    #Read image and mask
    file_image = path_image + '/' + image
    #im = plt.imread(file_image)
    im = Image.open(file_image) #I've been using this for asymm
    im_np = np.float16(im)

    file_mask = path_mask + '/mask_' + image
    #mask = plt.imread(file_mask)
    mask = Image.open(file_mask) #I've been using this for asymm
    mask_np = np.float16(mask)

    data['asym'].append(get_asymmetry(mask))
    data['area/perimeter'].append(area_perimeter(mask))
    data['compactness'].append(get_compactness(mask))

df = pd.DataFrame(data)
    
df
#ex_mask = Image.open(path_mask + '/mask_' + images_id[-1])
#fix_mask(ex_mask)

C:\Users\flore\AppData\Local\Temp\ipykernel_35568\1344857629.py:43: DeprecationWarning: FLIP_LEFT_RIGHT is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_LEFT_RIGHT instead.
  right = right.transpose(Image.FLIP_LEFT_RIGHT) #flip right part to compare


,images_id,asym,area/perimeter,compactness
0,PAT_26_37_865.png,0.075,"(38074, 709)",1.0506
1,PAT_38_54_234.png,0.103,"(167806, 1449)",0.9957
2,PAT_42_58_13.png,0.086,"(67711, 964)",1.0922
3,PAT_461_895_72.png,0.191,"(463197, 3523)",2.1323
4,PAT_46_881_939.png,0.121,"(481424, 2609)",1.1252
5,PAT_53_1370_245.png,0.179,"(136850, 1464)",1.2463
6,PAT_55_84_506.png,0.051,"(114141, 1131)",0.8918
7,PAT_59_94_849.png,0.105,"(211741, 1733)",1.1287
8,PAT_79_120_532.png,0.124,"(23758, 602)",1.2139
9,PAT_87_133_391.png,0.066,"(281, 54)",0.8258


In [16]:
''''  OK   '''

# Create an empty dataframe to store the features
features = pd.DataFrame()

# Create columns image_id, asymmetry, color_variability, and compactness
features['image_id'] = images_id
features['asymmetry'] = [get_asymmetry(Image.open(path_mask + '/mask_' + image)) for image in images_id]
features['color_variability'] = [get_color_variability(np.array(Image.open(path_image + '/' + image)), measure='variance') for image in images_id]
features['compactness'] = [get_compactness(Image.open(path_mask + '/mask_' + image)) for image in images_id]

C:\Users\raivi\AppData\Local\Temp\ipykernel_32536\3605390448.py:43: DeprecationWarning: FLIP_LEFT_RIGHT is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_LEFT_RIGHT instead.
  right = right.transpose(Image.FLIP_LEFT_RIGHT) #flip right part to compare


,image_id,asymmetry,color_variability,compactness
PAT_106_159_325.png,PAT_106_159_325.png,0.193,681.114949,1.2113
PAT_107_160_609.png,PAT_107_160_609.png,0.029,344.775174,0.8237
PAT_108_162_660.png,PAT_108_162_660.png,0.051,340.697442,1.0391
PAT_115_1138_870.png,PAT_115_1138_870.png,0.192,414.696603,1.3270
PAT_117_179_983.png,PAT_117_179_983.png,0.225,201.703397,1.6368
...,...,...,...,...
PAT_966_1825_584.png,PAT_966_1825_584.png,0.124,825.456553,1.7691
PAT_967_1827_247.png,PAT_967_1827_247.png,0.065,804.228394,0.9914
PAT_975_4734_783.png,PAT_975_4734_783.png,0.241,214.111459,2.2698
PAT_98_152_562.png,PAT_98_152_562.png,0.113,461.064668,1.0163


In [ ]:
features.to_csv('features.csv', index=False)

In [ ]:

columns = {'Image ID' : 'Asymmetry' : get_asymmetry(mask), 'Compactness' : get_compactness(mask), 'Color Variability' : get_color_variability(masked_image, measure='variance')}
feature_names = features.keys()
features_array = np.zeros([len(images_id),len(feature_names)], dtype=np.float16)

for i, image in enumerate(images_id):

    #Read image and mask
    file_image = path_image + '/' + image
    #im = plt.imread(file_image)
    im = Image.open(file_image) #I've been using this for asymm

    file_mask = path_mask + '/mask_' + image
    file_masked_image = path_masked_image + '/masked_' + image
    #mask = plt.imread(file_mask)
    mask = Image.open(file_mask) #I've been using this for asymm
    masked_image = io.imread(file_masked_image) #I've been using this for color variability

    #Measure features
    for j, feature in enumerate(feature_names):
        if feature == 'Color Variability':
            features_array[i,j] = get_color_variability(masked_image, measure='variance')
        elif feature == 'Asymmetry':
            features_array[i,j] = get_asymmetry(mask)
        elif feature == 'Compactness':
            features_array[i,j] = get_compactness(mask)


# Save features of each image to a csv file
file_features = path_features + '/features.csv'

#Save features to csv file
df_features = pd.DataFrame(features_array, columns = feature_names)
df_features.to_csv(file_features, index=False)

### Train classifiers

In [49]:
# Load the features - remember the example features are not informative
df_features = pd.read_csv(file_features)


# Make the dataset, you can select different classes (see task 0)
x = np.array(df_features[feature_names])
#y =  label == 'NEV'   #now True means healthy nevus, False means something else
patient_id = metadata['patient_id']

patient_id





0       PAT_1516
1         PAT_46
2       PAT_1545
3       PAT_1989
4        PAT_684
          ...   
2293    PAT_1708
2294      PAT_46
2295    PAT_1343
2296     PAT_326
2297    PAT_1714
Name: patient_id, Length: 2298, dtype: object

### Evaluate classifier

In [ ]:
'''This is still copy/paste Veronika's --- still need changes '''


# The function that should classify new images. 
# The image and mask are the same size, and are already loaded using plt.imread
def classify(img, mask):
    
    
     #Resize the image etc, if you did that during training
    
     #Extract features (the same ones that you used for training)
     x = extract_features(img, mask)
         
     
     #Load the trained classifier
     classifier = pickle.load(open('groupXY_classifier.sav', 'rb'))
    
    
     #Use it on this example to predict the label AND posterior probability
     pred_label = classifier.predict(x)
     pred_prob = classifier.predict_proba(x)
     
     
     #print('predicted label is ', pred_label)
     #print('predicted probability is ', pred_prob)
     return pred_label, pred_prob
 
    
# The TAs will call the function above in a loop, for external test images/masks